http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/apriori/

## Import Packages

In [1]:
import numpy as np
import pandas as pd
from functools import reduce
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from mlxtend.frequent_patterns import apriori
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from mlxtend.preprocessing import TransactionEncoder
%matplotlib inline

plt.rcParams["figure.figsize"] = (15,10)

## Import Data

In [2]:
df = pd.read_excel('IMData.xlsx')
df.head()

,SEC Region,Organization CRD#,SEC#,Legal Name,Main Office City,Main Office State,Main Office Country,Main Office Postal Code,Current Status,BOR Maintained Outside Principal Place of Business,...,Related Person - Banking or Thrift Institution,Related Person - Trust Company,Related Person - Accountant or Accounting Firm,Related Person - Lawyer or Law firm,Related Person - Insurance Company or Agency,Related Person - Pension Consultant,Related Person - Real Estate Broker/Dealer,Related Person - Sponsor/Syndicator or Limited Partnerships,Related Person - Sponsor/General Partner/Managing Member of PIVs,Private Fund Adviser?
0,MIRO,25,801-57874,"ALLEN & COMPANY OF FLORIDA, INC.",LAKELAND,FL,UNITED STATES,33803,Approved,Y,...,N,N,N,N,Y,N,N,N,N,N
1,SFRO,38,801-57838,"L.S.Y., INC",SAN RAMON,CA,UNITED STATES,94583,Approved,Y,...,N,N,Y,Y,Y,N,Y,Y,N,N
2,PLRO,53,801-55083,"HEFREN-TILLOTSON, INC.",PITTSBURGH,PA,UNITED STATES,15222,Approved,Y,...,N,N,N,N,N,N,N,N,N,N
3,CHRO,61,801-19699,B. C. ZIEGLER AND COMPANY,CHICAGO,IL,UNITED STATES,60606,Approved,Y,...,N,N,N,N,N,N,N,Y,N,Y
4,NYRO,70,801-56943,"BCG SECURITIES, INC.",DELRAN,NJ,UNITED STATES,8075,Approved,Y,...,N,N,N,N,N,Y,N,N,N,N


#### **NOTE:** Use on 2nd Pass / "features_to_use" is a list of columns that have been prioritized by Feature Importance

In [34]:
df = df[features_to_use]
df.head()

,AUM Type Banking/Thrift Institutions,AUM Type Insurance Companies,AUM Type Investment Companies,AUM Type Other,AUM Type Pension/Profit Sharing Plans,Advisory Services Type - Educational Seminars/Workshops,Advisory Services Type - Other,Advisory Services Type - Portfolio Mgmt for Investment Companies,Advisory Services Type - Portfolio Mgmt for PIV,Advisory Services Type - Selection of other Advisers,...,Provide Investment Advice wrt Limited Investment types,Public Reporting Company,Related Person - Banking or Thrift Institution,Related Person - Insurance Company or Agency,Related Person - Real Estate Broker/Dealer,Related Person - Registered Municipal Advisor,Related Person - Trust Company,SEC Region,Total Number of Accounts,Wrap Fee Program Portfolio Manager
0,Up to 25 percent,0 percent,0 percent,NaN,Up to 25 percent,N,N,N,N,Y,...,N,N,N,Y,N,Y,N,MIRO,1725,N
1,0 percent,Up to 25 percent,0 percent,Up to 25 percent,Up to 25 percent,N,N,N,N,Y,...,N,N,N,Y,Y,N,N,SFRO,1725,N
2,0 percent,0 percent,0 percent,0 percent,Up to 25 percent,N,N,N,N,N,...,N,N,N,N,N,N,N,PLRO,7754,N
3,0 percent,0 percent,0 percent,Up to 25 percent,Up to 25 percent,N,N,N,N,Y,...,N,Y,N,N,N,N,N,CHRO,1232,N
4,0 percent,0 percent,0 percent,NaN,Up to 25 percent,N,N,N,N,Y,...,Y,N,N,N,N,N,N,NYRO,755,N


## Exploratory Data Analysis

In [38]:
df.describe()

,Discretionary Number of Accounts,Investment advisory clients rounded to nearest 101,Non-Discretionary Number of Accounts,Number of Employees,Number of Employees performing IA Functions,Number of Employess that are B-D Reg Reps,Number of employees registered as IA reps with State Security Authorities,Number of employees that are licensed agents of insurance company,Number of firms or persons soliciting advisory clients on your behalf,Percentage of IA Clients that are non-US,Total Number of Accounts
count,2032.000000,2.032000e+03,2032.000000,2028.000000,2028.000000,2028.000000,2028.000000,2028.000000,2028.000000,2028.000000,2.032000e+03
mean,3734.954724,8.853347e+03,1802.863681,207.431953,100.004931,127.355523,89.934911,87.024655,5.336785,24.025148,5.537818e+03
std,37528.506266,1.909288e+05,26058.755790,1434.488783,827.203497,1060.539452,878.837417,809.789496,79.096667,38.424624,5.609630e+04
min,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00
25%,5.000000,0.000000e+00,0.000000,4.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000e+01
50%,55.000000,0.000000e+00,0.000000,11.000000,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,9.200000e+01
75%,442.750000,3.000000e+02,16.000000,41.000000,18.000000,3.000000,4.000000,0.000000,1.000000,50.000000,5.745000e+02
max,802747.000000,8.300000e+06,768136.000000,38700.000000,24700.000000,32900.000000,24700.000000,20897.000000,3000.000000,100.000000,1.349469e+06


In [39]:
df.shape

(2032, 47)

In [40]:
#df.info(verbose=True, null_counts=True)

In [41]:
headers = df.columns.values
#print(len(headers))
#sorted(headers)

## Data Preprocessing

#### Test Record

In [42]:
# Profile the first record to get the list of columns to clean
test_df = df[:1].T

#### 1. Free Response Columns (To be deleted)

In [43]:
del_cols = ['Organization CRD#', 'SEC#', 'Legal Name', 'Main Office City', 'Main Office Postal Code', 
            'Number of FP Services Clients in last FY Over 500', # Number columns have datetime values; too lazy to clean 
            'Number of FP Services Clients in last FY', 
            'Number of investment advisory clients as of last FY']

other_df = df.filter(regex='.+(Other\s.Description.)|(.+Specify)|(.+Description)', axis=1)

for col in list(other_df.columns.values):
    del_cols.append(col)

del_cols = sorted(del_cols)
del_cols

['Legal Name',
 'Main Office City',
 'Main Office Postal Code',
 'Number of FP Services Clients in last FY',
 'Number of FP Services Clients in last FY Over 500',
 'Number of investment advisory clients as of last FY',
 'Organization CRD#',
 'SEC#']

In [44]:
#df[del_cols].head()

#### 2. Columns with Boolean Values (Y/N/NULL)

In [45]:
yesno = (test_df[0]=='Y') | (test_df[0]=='N')

bool_cols = sorted(list(test_df[yesno].index))
bool_df = df[bool_cols]
bool_df.head()

,Advisory Services Type - Educational Seminars/Workshops,Advisory Services Type - Other,Advisory Services Type - Portfolio Mgmt for Investment Companies,Advisory Services Type - Portfolio Mgmt for PIV,Advisory Services Type - Selection of other Advisers,Compensation Type Fixed Fees,Compensation Type Performance Based Fees,IA or Sub-IA to an Investment Company,Non-Investment Advice Business?,Other Business Activities - Commodity Pool Operator/Commodity Trading Advisor,Other Business Activities - Trust Company,Private Fund Adviser?,Provide Investment Advice wrt Limited Investment types,Public Reporting Company,Related Person - Banking or Thrift Institution,Related Person - Insurance Company or Agency,Related Person - Real Estate Broker/Dealer,Related Person - Registered Municipal Advisor,Related Person - Trust Company,Wrap Fee Program Portfolio Manager
0,N,N,N,N,Y,N,N,N,N,N,N,N,N,N,N,Y,N,Y,N,N
1,N,N,N,N,Y,Y,N,N,N,N,N,N,N,N,N,Y,Y,N,N,N
2,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N
3,N,N,N,N,Y,Y,N,N,N,N,N,Y,N,Y,N,N,N,N,N,N
4,N,N,N,N,Y,Y,N,N,N,N,N,N,Y,N,N,N,N,N,N,N


In [46]:
# Instatiate the imputer
bool_imputer = SimpleImputer(strategy='constant', fill_value='N')

# Impute the NaNs with "N", if not "Y" it is implied "N"
bool_df = pd.DataFrame(bool_imputer.fit_transform(bool_df), columns=bool_df.columns)

In [47]:
# Change the values from Y/N to 1/0
bool_df = bool_df.replace({'Y':1, 'N':0})
bool_df.head()

,Advisory Services Type - Educational Seminars/Workshops,Advisory Services Type - Other,Advisory Services Type - Portfolio Mgmt for Investment Companies,Advisory Services Type - Portfolio Mgmt for PIV,Advisory Services Type - Selection of other Advisers,Compensation Type Fixed Fees,Compensation Type Performance Based Fees,IA or Sub-IA to an Investment Company,Non-Investment Advice Business?,Other Business Activities - Commodity Pool Operator/Commodity Trading Advisor,Other Business Activities - Trust Company,Private Fund Adviser?,Provide Investment Advice wrt Limited Investment types,Public Reporting Company,Related Person - Banking or Thrift Institution,Related Person - Insurance Company or Agency,Related Person - Real Estate Broker/Dealer,Related Person - Registered Municipal Advisor,Related Person - Trust Company,Wrap Fee Program Portfolio Manager
0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0
1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,1,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0
4,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0


#### 3. Columns with Continuous Variables

In [48]:
# Filter for numerical columns
num_cols = sorted(list(test_df[pd.to_numeric(test_df[0], errors='coerce').notnull()].index))
num_cols.append('Number of FP Services Clients in last FY Over 500')
num_cols = [x for x in num_cols if x not in del_cols]
num_cols = sorted(num_cols)

# Numerical columns dataframe
num_df = df[num_cols]
num_df.head()

,Discretionary Number of Accounts,Investment advisory clients rounded to nearest 101,Non-Discretionary Number of Accounts,Number of Employees,Number of Employees performing IA Functions,Number of Employess that are B-D Reg Reps,Number of employees registered as IA reps with State Security Authorities,Number of employees that are licensed agents of insurance company,Number of firms or persons soliciting advisory clients on your behalf,Percentage of IA Clients that are non-US,Total Number of Accounts
0,0,1700,1725,61.0,35.0,35.0,35.0,34.0,0.0,1.0,1725
1,1412,1000,313,46.0,46.0,46.0,44.0,35.0,1.0,0.0,1725
2,3740,5600,4014,111.0,3.0,102.0,65.0,66.0,0.0,1.0,7754
3,1095,1140,137,221.0,81.0,81.0,57.0,0.0,0.0,0.0,1232
4,55,600,700,75.0,30.0,70.0,25.0,40.0,0.0,0.0,755


In [49]:
# Instatiate the imputer
num_imputer = SimpleImputer(strategy='mean')

# Impute the NaNs with "N", if not "Y" it is implied "N"
num_df = pd.DataFrame(num_imputer.fit_transform(num_df), columns=num_df.columns)
num_df.head()

,Discretionary Number of Accounts,Investment advisory clients rounded to nearest 101,Non-Discretionary Number of Accounts,Number of Employees,Number of Employees performing IA Functions,Number of Employess that are B-D Reg Reps,Number of employees registered as IA reps with State Security Authorities,Number of employees that are licensed agents of insurance company,Number of firms or persons soliciting advisory clients on your behalf,Percentage of IA Clients that are non-US,Total Number of Accounts
0,0.0,1700.0,1725.0,61.0,35.0,35.0,35.0,34.0,0.0,1.0,1725.0
1,1412.0,1000.0,313.0,46.0,46.0,46.0,44.0,35.0,1.0,0.0,1725.0
2,3740.0,5600.0,4014.0,111.0,3.0,102.0,65.0,66.0,0.0,1.0,7754.0
3,1095.0,1140.0,137.0,221.0,81.0,81.0,57.0,0.0,0.0,0.0,1232.0
4,55.0,600.0,700.0,75.0,30.0,70.0,25.0,40.0,0.0,0.0,755.0


#### Binning the Numerical Columns to Convert to Categorical Columns

In [50]:
num_df.describe()

,Discretionary Number of Accounts,Investment advisory clients rounded to nearest 101,Non-Discretionary Number of Accounts,Number of Employees,Number of Employees performing IA Functions,Number of Employess that are B-D Reg Reps,Number of employees registered as IA reps with State Security Authorities,Number of employees that are licensed agents of insurance company,Number of firms or persons soliciting advisory clients on your behalf,Percentage of IA Clients that are non-US,Total Number of Accounts
count,2032.000000,2.032000e+03,2032.000000,2032.000000,2032.000000,2032.000000,2032.000000,2032.000000,2032.000000,2032.000000,2.032000e+03
mean,3734.954724,8.853347e+03,1802.863681,207.431953,100.004931,127.355523,89.934911,87.024655,5.336785,24.025148,5.537818e+03
std,37528.506266,1.909288e+05,26058.755790,1433.075493,826.388518,1059.494585,877.971567,808.991673,79.018739,38.386768,5.609630e+04
min,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00
25%,5.000000,0.000000e+00,0.000000,4.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000e+01
50%,55.000000,0.000000e+00,0.000000,11.000000,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,9.200000e+01
75%,442.750000,3.000000e+02,16.000000,42.000000,18.000000,3.000000,4.000000,0.000000,1.000000,50.000000,5.745000e+02
max,802747.000000,8.300000e+06,768136.000000,38700.000000,24700.000000,32900.000000,24700.000000,20897.000000,3000.000000,100.000000,1.349469e+06


In [51]:
# Code for auto-binning a column
#pd.qcut(Dataframe[Column].rank(method='first'), 3, labels=['Class_01', 'Class_02', 'Class_03'], duplicates='drop')

In [52]:
# Use an automated binning preprocessing step to automatically bin the numerical columns in to 3 categorical types
bin_num_df = pd.DataFrame()

for col in num_df.columns.values.tolist():
    bin_num_df[col] = pd.qcut(num_df[col].rank(method='first'), 3, labels=['Bin_01', 'Bin_02', 'Bin_03'], duplicates='drop')
    
bin_num_df.head()

,Discretionary Number of Accounts,Investment advisory clients rounded to nearest 101,Non-Discretionary Number of Accounts,Number of Employees,Number of Employees performing IA Functions,Number of Employess that are B-D Reg Reps,Number of employees registered as IA reps with State Security Authorities,Number of employees that are licensed agents of insurance company,Number of firms or persons soliciting advisory clients on your behalf,Percentage of IA Clients that are non-US,Total Number of Accounts
0,Bin_01,Bin_03,Bin_03,Bin_03,Bin_03,Bin_03,Bin_03,Bin_03,Bin_01,Bin_02,Bin_03
1,Bin_03,Bin_03,Bin_03,Bin_03,Bin_03,Bin_03,Bin_03,Bin_03,Bin_03,Bin_01,Bin_03
2,Bin_03,Bin_03,Bin_03,Bin_03,Bin_01,Bin_03,Bin_03,Bin_03,Bin_01,Bin_02,Bin_03
3,Bin_03,Bin_03,Bin_03,Bin_03,Bin_03,Bin_03,Bin_03,Bin_01,Bin_01,Bin_01,Bin_03
4,Bin_02,Bin_03,Bin_03,Bin_03,Bin_03,Bin_03,Bin_03,Bin_03,Bin_01,Bin_01,Bin_03


In [53]:
# Get dummies dataframe off of the auto-binned, preprocessed numerical columns
bin_num_dummies_df = pd.get_dummies(bin_num_df)
bin_num_dummies_df.head()

,Discretionary Number of Accounts_Bin_01,Discretionary Number of Accounts_Bin_02,Discretionary Number of Accounts_Bin_03,Investment advisory clients rounded to nearest 101_Bin_01,Investment advisory clients rounded to nearest 101_Bin_02,Investment advisory clients rounded to nearest 101_Bin_03,Non-Discretionary Number of Accounts_Bin_01,Non-Discretionary Number of Accounts_Bin_02,Non-Discretionary Number of Accounts_Bin_03,Number of Employees_Bin_01,...,Number of employees that are licensed agents of insurance company_Bin_03,Number of firms or persons soliciting advisory clients on your behalf_Bin_01,Number of firms or persons soliciting advisory clients on your behalf_Bin_02,Number of firms or persons soliciting advisory clients on your behalf_Bin_03,Percentage of IA Clients that are non-US_Bin_01,Percentage of IA Clients that are non-US_Bin_02,Percentage of IA Clients that are non-US_Bin_03,Total Number of Accounts_Bin_01,Total Number of Accounts_Bin_02,Total Number of Accounts_Bin_03
0,1,0,0,0,0,1,0,0,1,0,...,1,1,0,0,0,1,0,0,0,1
1,0,0,1,0,0,1,0,0,1,0,...,1,0,0,1,1,0,0,0,0,1
2,0,0,1,0,0,1,0,0,1,0,...,1,1,0,0,0,1,0,0,0,1
3,0,0,1,0,0,1,0,0,1,0,...,0,1,0,0,1,0,0,0,0,1
4,0,1,0,0,0,1,0,0,1,0,...,1,1,0,0,1,0,0,0,0,1


#### 4. Columns with Categorical Variables (i.e. Region, Country, Approved, etc.)

In [54]:
# String columns
raw_cat_cols = test_df[pd.to_numeric(test_df[0], errors='coerce').isnull()]
categorical_filter = (raw_cat_cols[0] != 'Y') & (raw_cat_cols[0] != 'N')
raw_cat_cols = sorted(list(raw_cat_cols[categorical_filter].index))
cat_cols = [x for x in raw_cat_cols if x not in del_cols]

# Categorical columns dataframe
cat_df = df[cat_cols]
cat_df.head()

,AUM Type Banking/Thrift Institutions,AUM Type Insurance Companies,AUM Type Investment Companies,AUM Type Other,AUM Type Pension/Profit Sharing Plans,Main Office State,Organization State,Percent Client Type Banking/Thrift Institutions,Percent Client Type High Net Worth Individuals,Percent Client Type Insurance Companies,Percent Client Type Investment Companies,Percent Client Type Other,Percent Client Type Pension/Profit Sharing Plans,Percent Client Type Pooled Investment Vehicles,Percent Client Type non High Net Worth Individuals,SEC Region
0,Up to 25 percent,0 percent,0 percent,NaN,Up to 25 percent,FL,FL,Up to 10 percent,Up to 10 percent,0 percent,0 percent,NaN,Up to 10 percent,0 percent,76-99 percent,MIRO
1,0 percent,Up to 25 percent,0 percent,Up to 25 percent,Up to 25 percent,CA,CA,0 percent,Up to 10 percent,Up to 10 percent,0 percent,Up to 10 percent,Up to 10 percent,0 percent,76-99 percent,SFRO
2,0 percent,0 percent,0 percent,0 percent,Up to 25 percent,PA,PA,0 percent,26-50 percent,0 percent,0 percent,0 percent,Up to 10 percent,0 percent,11-25 percent,PLRO
3,0 percent,0 percent,0 percent,Up to 25 percent,Up to 25 percent,IL,WI,0 percent,Up to 10 percent,0 percent,0 percent,Up to 10 percent,Up to 10 percent,Up to 10 percent,76-99 percent,CHRO
4,0 percent,0 percent,0 percent,NaN,Up to 25 percent,NJ,PA,0 percent,11-25 percent,0 percent,0 percent,NaN,Up to 10 percent,0 percent,26-50 percent,NYRO


In [55]:
# Instatiate the imputer
cat_imputer = SimpleImputer(strategy='most_frequent')

# Impute the NaNs with "N", if not "Y" it is implied "N"
cat_df = pd.DataFrame(cat_imputer.fit_transform(cat_df), columns=cat_df.columns)
cat_df.head()

,AUM Type Banking/Thrift Institutions,AUM Type Insurance Companies,AUM Type Investment Companies,AUM Type Other,AUM Type Pension/Profit Sharing Plans,Main Office State,Organization State,Percent Client Type Banking/Thrift Institutions,Percent Client Type High Net Worth Individuals,Percent Client Type Insurance Companies,Percent Client Type Investment Companies,Percent Client Type Other,Percent Client Type Pension/Profit Sharing Plans,Percent Client Type Pooled Investment Vehicles,Percent Client Type non High Net Worth Individuals,SEC Region
0,Up to 25 percent,0 percent,0 percent,0 percent,Up to 25 percent,FL,FL,Up to 10 percent,Up to 10 percent,0 percent,0 percent,0 percent,Up to 10 percent,0 percent,76-99 percent,MIRO
1,0 percent,Up to 25 percent,0 percent,Up to 25 percent,Up to 25 percent,CA,CA,0 percent,Up to 10 percent,Up to 10 percent,0 percent,Up to 10 percent,Up to 10 percent,0 percent,76-99 percent,SFRO
2,0 percent,0 percent,0 percent,0 percent,Up to 25 percent,PA,PA,0 percent,26-50 percent,0 percent,0 percent,0 percent,Up to 10 percent,0 percent,11-25 percent,PLRO
3,0 percent,0 percent,0 percent,Up to 25 percent,Up to 25 percent,IL,WI,0 percent,Up to 10 percent,0 percent,0 percent,Up to 10 percent,Up to 10 percent,Up to 10 percent,76-99 percent,CHRO
4,0 percent,0 percent,0 percent,0 percent,Up to 25 percent,NJ,PA,0 percent,11-25 percent,0 percent,0 percent,0 percent,Up to 10 percent,0 percent,26-50 percent,NYRO


In [56]:
# Get dummies dataframe
cat_dummies_df = pd.get_dummies(cat_df)
cat_dummies_df.head()

,AUM Type Banking/Thrift Institutions_0 percent,AUM Type Banking/Thrift Institutions_More than 75 percent,AUM Type Banking/Thrift Institutions_Up to 25 percent,AUM Type Banking/Thrift Institutions_Up to 50 percent,AUM Type Banking/Thrift Institutions_Up to 75 percent,AUM Type Insurance Companies_0 percent,AUM Type Insurance Companies_More than 75 percent,AUM Type Insurance Companies_Up to 25 percent,AUM Type Insurance Companies_Up to 50 percent,AUM Type Insurance Companies_Up to 75 percent,...,SEC Region_BRO,SEC Region_CHRO,SEC Region_DRO,SEC Region_FWRO,SEC Region_HQ,SEC Region_LARO,SEC Region_MIRO,SEC Region_NYRO,SEC Region_PLRO,SEC Region_SFRO
0,0,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
2,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1,0,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


### Final Dataframe

In [23]:
# List of dataframes
dfs = [bool_df, cat_dummies_df, num_df]

# Final dataframe
df_final = reduce(lambda left, right: pd.merge(left, right, right_index=True, left_index=True), dfs)
df_final.head()

,Advisory Services Type - Educational Seminars/Workshops,Advisory Services Type - Financial Planning,Advisory Services Type - Market Timing Services,Advisory Services Type - Other,Advisory Services Type - Pension Consulting Services,Advisory Services Type - Periodicals/Newsletter Publications,Advisory Services Type - Portfolio Mgmt for Businesses,Advisory Services Type - Portfolio Mgmt for Individuals/SB,Advisory Services Type - Portfolio Mgmt for Investment Companies,Advisory Services Type - Portfolio Mgmt for PIV,...,Number of Employees,Number of Employees performing IA Functions,Number of Employess that are B-D Reg Reps,Number of employees registered as IA reps with State Security Authorities,Number of employees registered as IA reps with State Security Authorities for other IA Firms,Number of employees that are licensed agents of insurance company,Number of firms or persons soliciting advisory clients on your behalf,Percentage of IA Clients that are non-US,Total AUM,Total Number of Accounts
0,0,1,0,0,1,0,1,1,0,0,...,61.0,35.0,35.0,35.0,0.0,34.0,0.0,1.0,4.971368e+08,1725.0
1,0,1,0,0,0,0,1,1,0,0,...,46.0,46.0,46.0,44.0,6.0,35.0,1.0,0.0,2.574490e+08,1725.0
2,0,1,0,0,0,0,1,1,0,0,...,111.0,3.0,102.0,65.0,0.0,66.0,0.0,1.0,4.132077e+09,7754.0
3,0,1,0,0,0,0,1,1,0,0,...,221.0,81.0,81.0,57.0,0.0,0.0,0.0,0.0,3.353288e+08,1232.0
4,0,1,0,0,1,0,0,1,0,0,...,75.0,30.0,70.0,25.0,10.0,40.0,0.0,0.0,2.068060e+08,755.0


#### Delete Columns

In [24]:
# Don't run on 2nd pass
df_final = df_final.drop(['Discretionary AUM', 'Non-Discretionary AUM'], axis=1)
df_final.head()

,Advisory Services Type - Educational Seminars/Workshops,Advisory Services Type - Financial Planning,Advisory Services Type - Market Timing Services,Advisory Services Type - Other,Advisory Services Type - Pension Consulting Services,Advisory Services Type - Periodicals/Newsletter Publications,Advisory Services Type - Portfolio Mgmt for Businesses,Advisory Services Type - Portfolio Mgmt for Individuals/SB,Advisory Services Type - Portfolio Mgmt for Investment Companies,Advisory Services Type - Portfolio Mgmt for PIV,...,Number of Employees,Number of Employees performing IA Functions,Number of Employess that are B-D Reg Reps,Number of employees registered as IA reps with State Security Authorities,Number of employees registered as IA reps with State Security Authorities for other IA Firms,Number of employees that are licensed agents of insurance company,Number of firms or persons soliciting advisory clients on your behalf,Percentage of IA Clients that are non-US,Total AUM,Total Number of Accounts
0,0,1,0,0,1,0,1,1,0,0,...,61.0,35.0,35.0,35.0,0.0,34.0,0.0,1.0,4.971368e+08,1725.0
1,0,1,0,0,0,0,1,1,0,0,...,46.0,46.0,46.0,44.0,6.0,35.0,1.0,0.0,2.574490e+08,1725.0
2,0,1,0,0,0,0,1,1,0,0,...,111.0,3.0,102.0,65.0,0.0,66.0,0.0,1.0,4.132077e+09,7754.0
3,0,1,0,0,0,0,1,1,0,0,...,221.0,81.0,81.0,57.0,0.0,0.0,0.0,0.0,3.353288e+08,1232.0
4,0,1,0,0,1,0,0,1,0,0,...,75.0,30.0,70.0,25.0,10.0,40.0,0.0,0.0,2.068060e+08,755.0


In [25]:
df_final_columns = df_final.columns.values.tolist()
#df_final_columns

## Feature Selection

#### Select the most important Boolean variables

In [26]:
bool_df['Total AUM'] = df['Total AUM']
bool_df.head()

,Advisory Services Type - Educational Seminars/Workshops,Advisory Services Type - Financial Planning,Advisory Services Type - Market Timing Services,Advisory Services Type - Other,Advisory Services Type - Pension Consulting Services,Advisory Services Type - Periodicals/Newsletter Publications,Advisory Services Type - Portfolio Mgmt for Businesses,Advisory Services Type - Portfolio Mgmt for Individuals/SB,Advisory Services Type - Portfolio Mgmt for Investment Companies,Advisory Services Type - Portfolio Mgmt for PIV,...,Related Person - Real Estate Broker/Dealer,Related Person - Registered Municipal Advisor,Related Person - Registered security-based swap dealer,Related Person - Sponsor/General Partner/Managing Member of PIVs,Related Person - Sponsor/Syndicator or Limited Partnerships,Related Person - Trust Company,Sell non-IA products or services to advisory clients?,Wrap Fee Program Portfolio Manager,Wrap Fee Program Sponsor,Total AUM
0,0,1,0,0,1,0,1,1,0,0,...,0,1,0,0,0,0,0,0,0,497136850
1,0,1,0,0,0,0,1,1,0,0,...,1,0,0,0,1,0,1,0,0,257449038
2,0,1,0,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,1,4132076947
3,0,1,0,0,0,0,1,1,0,0,...,0,0,0,0,1,0,0,0,0,335328796
4,0,1,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,206806000


#### Select the most important Categorical variables

In [27]:
cat_dummies_df['Total AUM'] = df['Total AUM']
cat_dummies_df.head()

,AUM Type Banking/Thrift Institutions_0 percent,AUM Type Banking/Thrift Institutions_More than 75 percent,AUM Type Banking/Thrift Institutions_Up to 25 percent,AUM Type Banking/Thrift Institutions_Up to 50 percent,AUM Type Banking/Thrift Institutions_Up to 75 percent,AUM Type Business Development Companies_0 percent,AUM Type Business Development Companies_Up to 25 percent,AUM Type Business Development Companies_Up to 50 percent,AUM Type Charitable Organizations_0 percent,AUM Type Charitable Organizations_More than 75 percent,...,SEC Region_CHRO,SEC Region_DRO,SEC Region_FWRO,SEC Region_HQ,SEC Region_LARO,SEC Region_MIRO,SEC Region_NYRO,SEC Region_PLRO,SEC Region_SFRO,Total AUM
0,0,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,497136850
1,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,257449038
2,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,4132076947
3,1,0,0,0,0,1,0,0,1,0,...,1,0,0,0,0,0,0,0,0,335328796
4,1,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,1,0,0,206806000


#### Select the most important Numerical variables

In [28]:
num_aum_df = num_df.drop(['Discretionary AUM', 'Non-Discretionary AUM'], axis=1)
num_aum_df.head()

,Discretionary Number of Accounts,Investment advisory clients rounded to nearest 101,Non-Discretionary Number of Accounts,Number of Employees,Number of Employees performing IA Functions,Number of Employess that are B-D Reg Reps,Number of employees registered as IA reps with State Security Authorities,Number of employees registered as IA reps with State Security Authorities for other IA Firms,Number of employees that are licensed agents of insurance company,Number of firms or persons soliciting advisory clients on your behalf,Percentage of IA Clients that are non-US,Total AUM,Total Number of Accounts
0,0.0,1700.0,1725.0,61.0,35.0,35.0,35.0,0.0,34.0,0.0,1.0,4.971368e+08,1725.0
1,1412.0,1000.0,313.0,46.0,46.0,46.0,44.0,6.0,35.0,1.0,0.0,2.574490e+08,1725.0
2,3740.0,5600.0,4014.0,111.0,3.0,102.0,65.0,0.0,66.0,0.0,1.0,4.132077e+09,7754.0
3,1095.0,1140.0,137.0,221.0,81.0,81.0,57.0,0.0,0.0,0.0,0.0,3.353288e+08,1232.0
4,55.0,600.0,700.0,75.0,30.0,70.0,25.0,10.0,40.0,0.0,0.0,2.068060e+08,755.0


#### Data Preprocessing

In [29]:
# Independent/Dependent Variables
prices = df_final['Total AUM']
features = df_final.drop('Total AUM', axis=1)

# Scale the features
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)

# Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(scaled_features, prices, test_size=0.3, random_state=0)

C:\Users\alexcheng2\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\alexcheng2\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


#### Random Forest Regressor

In [30]:
# # Instantiate the regressor
# clf = RandomForestRegressor()

# # Fit the model
# clf.fit(X_train, y_train)

# # Score the model
# clf.score(X_test, y_test)

In [31]:
# Instantiate the regressor
clf = RandomForestRegressor()

# Fit the model
clf.fit(scaled_features, prices)

# Score the model
clf.score(scaled_features, prices)

C:\Users\alexcheng2\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.8231701479979456

### Feature Importance

In [32]:
feature_importances = clf.feature_importances_
headers = features.columns.values

data = {"Features":headers, "Feature Importances":feature_importances}

features_df = pd.DataFrame(data, columns=['Features', 'Feature Importances'])
features_df = features_df.sort_values(by='Feature Importances', ascending=False).reset_index(drop=True)
features_df.head(20)

,Features,Feature Importances
0,Number of Employees,0.159979
1,Percent Client Type Other_76-99 percent,0.133247
2,Number of Employees performing IA Functions,0.060162
3,SEC Region_LARO,0.057196
4,Public Reporting Company,0.053035
5,IA or Sub-IA to an Investment Company,0.048410
6,Percent Client Type Investment Companies_76-99...,0.033951
7,Organization State_DE,0.032762
8,Other Business Activities - Commodity Pool Ope...,0.030351
9,Total Number of Accounts,0.025929


In [33]:
# Create a list of features from the dataframe (300 rows creates about 100 columns to use / 55 rows creates about 40 cols)
list_features = sorted(list(features_df['Features'].head(55)))

# Create a list of columns to use
features_to_use = []

for col in list_features:
    features_to_use.append(col.split("_", 1)[0])
    
# Remove duplicate columns
features_to_use = list(dict.fromkeys(features_to_use))

# Print the number of columns 
print("{} columns".format(len(features_to_use)))
print()
features_to_use

47 columns



['AUM  Type Banking/Thrift Institutions',
 'AUM  Type Insurance Companies',
 'AUM  Type Investment Companies',
 'AUM  Type Other',
 'AUM  Type Pension/Profit Sharing Plans',
 'Advisory Services Type - Educational Seminars/Workshops',
 'Advisory Services Type - Other',
 'Advisory Services Type - Portfolio Mgmt for Investment Companies',
 'Advisory Services Type - Portfolio Mgmt for PIV',
 'Advisory Services Type - Selection of other Advisers',
 'Compensation Type Fixed Fees',
 'Compensation Type Performance Based Fees',
 'Discretionary Number of Accounts',
 'IA or Sub-IA to an Investment Company',
 'Investment advisory clients rounded to nearest 101',
 'Main Office State',
 'Non-Discretionary Number of Accounts',
 'Non-Investment Advice Business?',
 'Number of Employees',
 'Number of Employees performing IA Functions',
 'Number of Employess that are B-D Reg Reps',
 'Number of employees registered as IA reps with State Security Authorities',
 'Number of employees that are licensed agents

### Run On 2nd Pass, Prepare the Final Dataframe for Apriori w/ Prioritized List of Columns

In [57]:
# List of dataframes
dfs = [bool_df, cat_dummies_df, bin_num_dummies_df]

# Final dataframe
prep_df = reduce(lambda left, right: pd.merge(left, right, right_index=True, left_index=True), dfs)
prep_df.head()

,Advisory Services Type - Educational Seminars/Workshops,Advisory Services Type - Other,Advisory Services Type - Portfolio Mgmt for Investment Companies,Advisory Services Type - Portfolio Mgmt for PIV,Advisory Services Type - Selection of other Advisers,Compensation Type Fixed Fees,Compensation Type Performance Based Fees,IA or Sub-IA to an Investment Company,Non-Investment Advice Business?,Other Business Activities - Commodity Pool Operator/Commodity Trading Advisor,...,Number of employees that are licensed agents of insurance company_Bin_03,Number of firms or persons soliciting advisory clients on your behalf_Bin_01,Number of firms or persons soliciting advisory clients on your behalf_Bin_02,Number of firms or persons soliciting advisory clients on your behalf_Bin_03,Percentage of IA Clients that are non-US_Bin_01,Percentage of IA Clients that are non-US_Bin_02,Percentage of IA Clients that are non-US_Bin_03,Total Number of Accounts_Bin_01,Total Number of Accounts_Bin_02,Total Number of Accounts_Bin_03
0,0,0,0,0,1,0,0,0,0,0,...,1,1,0,0,0,1,0,0,0,1
1,0,0,0,0,1,1,0,0,0,0,...,1,0,0,1,1,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,1,0,0,0,1
3,0,0,0,0,1,1,0,0,0,0,...,0,1,0,0,1,0,0,0,0,1
4,0,0,0,0,1,1,0,0,0,0,...,1,1,0,0,1,0,0,0,0,1


In [59]:
# Unnecessary to re-reun in 2nd pass
# prep_df = prep_df.drop(['Total AUM_x', 'Total AUM_y'], axis=1)
# prep_df.head()

In [60]:
# # Create a list of columns to use
# prep_df_columns = []

# for col in prep_df.columns.values.tolist():
#     prep_df_columns.append(col.split("_", 1)[0])
    
# # Remove duplicate columns
# prep_df_columns = sorted(list(dict.fromkeys(prep_df_columns)))
# len(prep_df_columns)

### Apriori / Association Rule Mining

In [61]:
prep_df.head()

,Advisory Services Type - Educational Seminars/Workshops,Advisory Services Type - Other,Advisory Services Type - Portfolio Mgmt for Investment Companies,Advisory Services Type - Portfolio Mgmt for PIV,Advisory Services Type - Selection of other Advisers,Compensation Type Fixed Fees,Compensation Type Performance Based Fees,IA or Sub-IA to an Investment Company,Non-Investment Advice Business?,Other Business Activities - Commodity Pool Operator/Commodity Trading Advisor,...,Number of employees that are licensed agents of insurance company_Bin_03,Number of firms or persons soliciting advisory clients on your behalf_Bin_01,Number of firms or persons soliciting advisory clients on your behalf_Bin_02,Number of firms or persons soliciting advisory clients on your behalf_Bin_03,Percentage of IA Clients that are non-US_Bin_01,Percentage of IA Clients that are non-US_Bin_02,Percentage of IA Clients that are non-US_Bin_03,Total Number of Accounts_Bin_01,Total Number of Accounts_Bin_02,Total Number of Accounts_Bin_03
0,0,0,0,0,1,0,0,0,0,0,...,1,1,0,0,0,1,0,0,0,1
1,0,0,0,0,1,1,0,0,0,0,...,1,0,0,1,1,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,1,0,0,0,1
3,0,0,0,0,1,1,0,0,0,0,...,0,1,0,0,1,0,0,0,0,1
4,0,0,0,0,1,1,0,0,0,0,...,1,1,0,0,1,0,0,0,0,1


In [62]:
x = True
y = False
print(bool(x))

True


In [63]:
prep_df = prep_df.replace({1:x, 0:y})
prep_df.head()

,Advisory Services Type - Educational Seminars/Workshops,Advisory Services Type - Other,Advisory Services Type - Portfolio Mgmt for Investment Companies,Advisory Services Type - Portfolio Mgmt for PIV,Advisory Services Type - Selection of other Advisers,Compensation Type Fixed Fees,Compensation Type Performance Based Fees,IA or Sub-IA to an Investment Company,Non-Investment Advice Business?,Other Business Activities - Commodity Pool Operator/Commodity Trading Advisor,...,Number of employees that are licensed agents of insurance company_Bin_03,Number of firms or persons soliciting advisory clients on your behalf_Bin_01,Number of firms or persons soliciting advisory clients on your behalf_Bin_02,Number of firms or persons soliciting advisory clients on your behalf_Bin_03,Percentage of IA Clients that are non-US_Bin_01,Percentage of IA Clients that are non-US_Bin_02,Percentage of IA Clients that are non-US_Bin_03,Total Number of Accounts_Bin_01,Total Number of Accounts_Bin_02,Total Number of Accounts_Bin_03
0,False,False,False,False,True,False,False,False,False,False,...,True,True,False,False,False,True,False,False,False,True
1,False,False,False,False,True,True,False,False,False,False,...,True,False,False,True,True,False,False,False,False,True
2,False,False,False,False,False,False,False,False,False,False,...,True,True,False,False,False,True,False,False,False,True
3,False,False,False,False,True,True,False,False,False,False,...,False,True,False,False,True,False,False,False,False,True
4,False,False,False,False,True,True,False,False,False,False,...,True,True,False,False,True,False,False,False,False,True


In [ ]:
# cat_dummies_df = cat_dummies_df.replace({1:x, 0:y})
# cat_dummies_df.head()

In [ ]:
# bool_cat = pd.merge(bool_df, cat_dummies_df, right_index=True, left_index=True)
# bool_cat.head()

In [64]:
prep_df.shape

(2032, 243)

In [ ]:
#prep_df.to_excel('test.xlsx')

### Frequent Itemsets with the Boolean Columns

In [65]:
apriori_df = apriori(prep_df, min_support=0.1, use_colnames=True).sort_values(by='support', ascending=False).reset_index(drop=True)
apriori_df.head()

,support,itemsets
0,0.899114,(AUM Type Banking/Thrift Institutions_0 percent)
1,0.886811,(Percent Client Type Banking/Thrift Institutio...
2,0.884350,(Percent Client Type Banking/Thrift Institutio...
3,0.877461,(AUM Type Insurance Companies_0 percent)
4,0.870571,(AUM Type Other_0 percent)


In [66]:
apriori_df['length'] = apriori_df['itemsets'].apply(lambda x: len(x))
apriori_df = apriori_df[ (apriori_df['length'] >= 3) & (apriori_df['support'] >= 0.5) ]
apriori_df = apriori_df.reset_index(drop=True)

In [67]:
apriori_df.describe()

,support,length
count,323.000000,323.000000
mean,0.626176,4.263158
std,0.078193,1.084377
min,0.500000,3.000000
25%,0.554380,3.000000
50%,0.627461,4.000000
75%,0.674705,5.000000
max,0.816437,8.000000


In [68]:
apriori_df.to_excel("IMData_association_rules_47columns.xlsx")

### Working with Sparse Representations

In [ ]:
# te = TransactionEncoder()

# te_boolcat = te.fit(bool_cat).transform(bool_cat, sparse=True)
# sparse_df = pd.SparseDataFrame(te_boolcat, columns=te.columns_, default_fill_value=False)
# sparse_df

### Titanic Example

In [ ]:
ex_df = pd.read_csv("titanic.csv")

#### Age

In [ ]:
ex_df['Age'].head()

In [ ]:
ex_df['Age'].unique()

In [ ]:
def age_imputer(x):
    if x < 20:
        return 'youth'
    elif 21 < x < 50:
        return 'adult'
    else:
        return 'elderly'

In [ ]:
ex_df['Age'] = ex_df['Age'].apply(age_imputer)

In [ ]:
Age_dummies = pd.get_dummies(ex_df['Age'], prefix='Age', drop_first=True)
Age_dummies.head()

#### Sex

In [ ]:
# Create an encoder
sex_encoder = preprocessing.LabelEncoder()

In [ ]:
ex_df['Sex'] = sex_encoder.fit_transform(ex_df['Sex'])

In [ ]:
ex_df['Sex'].head()

In [ ]:
print("There are {}% males".format(ex_df['Sex'].sum() / ex_df['Sex'].count()))

#### Passenger Class

In [ ]:
# Convert the Passenger class feature into dummy variables using one-hot and leave one first category
Pclass_dummies = pd.get_dummies(ex_df['Pclass'], prefix='Pclass', drop_first=True)
Pclass_dummies.head()

In [ ]:
pclass_pivot = pd.pivot_table(ex_df, index='Pclass', values='Name', aggfunc='count', margins=True, margins_name='Total')
pclass_pivot['Percent'] = pclass_pivot['Name'] / pclass_pivot.Name.Total
pclass_pivot

#### Survived

In [ ]:
ex_df['Survived'].unique()

#### Siblings/Spouses

In [ ]:
ex_df['Siblings/Spouses Aboard'].unique()

#### Parents/Children Aboard

In [ ]:
ex_df['Parents/Children Aboard'].unique()

### Data Preprocessing

In [ ]:
data = ex_df[['Survived', 'Pclass', 'Sex', 'Age']]
data = data.rename(columns={'Survived':'Survived_yes', 'Sex':'Sex_male'})
data = pd.concat([data, Pclass_dummies, Age_dummies], axis=1)
data = data.drop(['Pclass', 'Age'], axis=1)
data.head()

In [ ]:
x = True
y = False
print(bool(x))

In [ ]:
data = data.replace({1:x, 0:y})
data.head()

In [ ]:
data.info()

### Apriori / Association Rule Mining 

- **Support** is an indication of how frequently the itemset appears in the dataset.

In [ ]:
apriori_titanic_df = apriori(data, min_support=0.1, use_colnames=True).sort_values(by='support', ascending=False).reset_index(drop=True)
apriori_titanic_df

#### Filtering Itemsets

In [ ]:
apriori_titanic_df['length'] = apriori_titanic_df['itemsets'].apply(lambda x: len(x))
apriori_titanic_df[ (apriori_titanic_df['length'] >= 2) & (apriori_titanic_df['support'] >= 0.1) ]